# Script 1 — Configuração Geral e Importações  
Carrega as bibliotecas e define os caminhos dos arquivos.  
Tudo que o projeto usa começa daqui.

In [5]:
import pandas as pd
import numpy as np
import re
import os

pd.set_option('display.max_columns', None)

# Caminhos dos arquivos (tudo na mesma pasta)
path_telemetria = "telemetria_detalhada_30dias.csv"
path_producao   = "historico_producao_1ano.csv"
path_eventos    = "eventos_industriais.csv"

print("Configurações carregadas.")

Configurações carregadas.


# Script 2 — Processamento da Telemetria  
Aqui faço a limpeza das leituras da máquina, padronizo temperatura, arrumo datas  
e gero a versão “Silver” dos dados de telemetria.

In [6]:
print("Processando telemetria...")

telemetria_df = pd.read_csv(path_telemetria)

# Converte tempo
telemetria_df["timestamp"] = pd.to_datetime(telemetria_df["timestamp"], errors="coerce")

# Função pra limpar as temperaturas bagunçadas
def clean_temp(x):
    if pd.isna(x): 
        return None
    x = re.sub(r"[^0-9\\.,-]", "", str(x)).replace(",", ".")
    try:
        return float(x)
    except:
        return None

telemetria_df["temp_matriz_c"] = telemetria_df["temp_matriz_c"].apply(clean_temp)

# Preenche falhas com último valor válido
telemetria_df["temp_matriz_c"] = telemetria_df["temp_matriz_c"].ffill()

# Ordenação pra organizar
telemetria_df = telemetria_df.sort_values(["maquina_id", "timestamp"]).reset_index(drop=True)

telemetria_df.to_csv("telemetria_silver.csv", index=False)

print("Telemetria Silver gerada.")
display(telemetria_df.head())
telemetria_df.info()

Processando telemetria...
Telemetria Silver gerada.


,timestamp,maquina_id,ciclo_tempo_s,pressao_mpa,umidade_pct,temp_matriz_c,pecas_produzidas,flag_defeito
0,2025-10-20 10:54:19.143015,1,7.19,16.47,11.92,57.0,2,0
1,2025-10-20 10:54:26.143015,1,7.23,15.26,11.28,59.1,2,0
2,2025-10-20 10:54:33.143015,1,7.11,14.29,9.91,58.8,2,1
3,2025-10-20 10:54:40.143015,1,7.09,14.22,12.09,55.0,2,1
4,2025-10-20 10:54:47.143015,1,7.46,15.34,11.32,60.0,2,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648000 entries, 0 to 647999
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   timestamp         648000 non-null  datetime64[ns]
 1   maquina_id        648000 non-null  int64         
 2   ciclo_tempo_s     648000 non-null  float64       
 3   pressao_mpa       648000 non-null  float64       
 4   umidade_pct       648000 non-null  float64       
 5   temp_matriz_c     648000 non-null  float64       
 6   pecas_produzidas  648000 non-null  int64         
 7   flag_defeito      648000 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 39.6 MB


# Script 3 — Processamento da Produção  
Calculo taxa de refugo, padronizo sinalizações, arrumo datas  
e gero a versão “Silver” dos dados de produção.

In [7]:
print("Processando produção...")

producao_df = pd.read_csv(path_producao)

# Converte tempo
producao_df["timestamp"] = pd.to_datetime(producao_df["timestamp"], errors="coerce")

# Taxa de refugo (tratando divisão por zero)
producao_df["taxa_refugo"] = producao_df["pecas_refugadas"] / producao_df["pecas_produzidas"]
producao_df["taxa_refugo"] = producao_df["taxa_refugo"].replace([np.inf, -np.inf], 0).fillna(0)

# Indica se teve produção
producao_df["produziu_algo"] = (producao_df["pecas_produzidas"] > 0).astype(int)

# Organização
producao_df = producao_df.sort_values(["maquina_id", "timestamp"]).reset_index(drop=True)

producao_df.to_csv("producao_silver.csv", index=False)

print("Produção Silver gerada.")
display(producao_df.head())
producao_df.info()

Processando produção...
Produção Silver gerada.


,timestamp,maquina_id,turno,pecas_produzidas,pecas_refugadas,consumo_kwh,status,taxa_refugo,produziu_algo
0,2024-11-19 10:54:19.143015,1,Manhã,937,16,9.709,Operando,0.017076,1
1,2024-11-19 11:54:19.143015,1,Manhã,956,20,10.151,Operando,0.020921,1
2,2024-11-19 12:54:19.143015,1,Manhã,928,11,9.931,Operando,0.011853,1
3,2024-11-19 13:54:19.143015,1,Manhã,905,15,9.057,Operando,0.016575,1
4,2024-11-19 14:54:19.143015,1,Tarde,928,21,9.965,Operando,0.022629,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17522 entries, 0 to 17521
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   timestamp         17522 non-null  datetime64[ns]
 1   maquina_id        17522 non-null  int64         
 2   turno             17522 non-null  object        
 3   pecas_produzidas  17522 non-null  int64         
 4   pecas_refugadas   17522 non-null  int64         
 5   consumo_kwh       17522 non-null  float64       
 6   status            17522 non-null  object        
 7   taxa_refugo       17522 non-null  float64       
 8   produziu_algo     17522 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(1), int64(3), object(2)
memory usage: 1.1+ MB


# Script 4 — Processamento dos Eventos Industriais  
Padronizo o texto dos eventos, converto severidade para número  
e gero a versão “Silver” dos eventos da fábrica.

In [ ]:
print("Processando eventos...")

eventos_df = pd.read_csv(path_eventos)

# Converte tempo
eventos_df["timestamp"] = pd.to_datetime(eventos_df["timestamp"], errors="coerce")

# Ajustes básicos de texto
eventos_df["evento"] = eventos_df["evento"].str.strip().str.title()
eventos_df["severidade"] = eventos_df["severidade"].str.strip().str.capitalize()
eventos_df["origem"] = eventos_df["origem"].str.strip().str.capitalize()

# Mapeia severidade pra número
map_sev = {"Baixa": 1, "Média": 2, "Alta": 3}
eventos_df["sev_codigo"] = eventos_df["severidade"].map(map_sev).fillna(0).astype(int)

# Se existir alguma coluna duplicada de severidade numérica, remove
if "sev_code" in eventos_df.columns:
    eventos_df = eventos_df.drop(columns=["sev_code"])

# Ordenação
eventos_df = eventos_df.sort_values(["maquina_id", "timestamp"]).reset_index(drop=True)

eventos_df.to_csv("eventos_silver.csv", index=False)

print("Eventos Silver gerados.")
display(eventos_df.head())
eventos_df.info()

Processando eventos...
Eventos Silver gerados.


,timestamp,maquina_id,evento,severidade,origem,sev_codigo
0,2024-11-20 17:54:19.143015,1,Falha De Motor,Baixa,Sensor,1
1,2024-11-22 16:54:19.143015,1,Sobrecarga Elétrica,Alta,Scada,3
2,2024-11-27 03:54:19.143015,1,Sobrecarga Elétrica,Baixa,Sensor,1
3,2024-11-27 15:54:19.143015,1,Parada Programada,Média,Operador,2
4,2024-11-27 19:54:19.143015,1,Falha De Motor,Baixa,Operador,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   timestamp   200 non-null    datetime64[ns]
 1   maquina_id  200 non-null    int64         
 2   evento      200 non-null    object        
 3   severidade  200 non-null    object        
 4   origem      200 non-null    object        
 5   sev_codigo  200 non-null    int32         
dtypes: datetime64[ns](1), int32(1), int64(1), object(3)
memory usage: 8.7+ KB


: 